<a href="https://colab.research.google.com/github/sanAkel/ufs_diurnal_diagnostics/blob/main/RTOFS/binary_nc_converter/read_archive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cartopy

In [ ]:
import xarray as xr
import numpy as np

import cartopy.crs as ccrs
import cartopy.feature as cfeature

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# mount drive
from google.colab import drive
drive.mount('/content/drive')

data_path="/content/drive/MyDrive/datasets/tmp/read_archive/"

In [ ]:
!ls -al $data_path

### These functions are based on those at [pyhycom.](https://github.com/uwincm/pyhycom/blob/master/pyhycom.py)

In [ ]:
# return file handle
def open_a_file(filename, mode):
    file = open(filename[:-1]+'a',mode=mode)
    return file

#Return the name of the corresponding HYCOM "b" file.
def get_b_filename(fName):
    bfilename = fName[:-1]+'b'
    return bfilename

#Return a list where each element contains text from each line of `b file`
def getTextFile(fName):
    return [line.rstrip() for line in open(fName,'r').readlines()]

In [ ]:
# get dimensions of an archive from .b file
def getDims(fName, topo_file=False):

  f = getTextFile(get_b_filename(fName))
  idmFound, jdmFound = [False, False]

  if topo_file:
    for line in f:
        if 'i/jdm' in line:
          xx = line.split()[3]; jdm = xx[0:4]
          idm = line.split()[2]
          idmFound = True
          jdmFound = True
        if idmFound and jdmFound:break
  else:
    for line in f:
        if 'idm' in line:
          idm = line.split()[0]
          idmFound = True
        if 'jdm' in line:
          jdm = line.split()[0]
          jdmFound = True
        if idmFound and jdmFound:break

  return int(jdm), int(idm)

In [ ]:
def getFieldIndex(field, fName):
    f = getTextFile(get_b_filename(fName))
    if 'arch' in fName.split('/')[-1]:f = f[10:] # skip first 10 lines
    if 'grid' in fName.split('/')[-1]:f = f[3:] # skip first 3 lines
    fieldIndex = []
    for line in f:
      if field == line.split()[0].replace('.','').replace(':',''):
        fieldIndex.append(f.index(line))
    return fieldIndex

In [ ]:
def getField(field, fName, undef=np.nan, x_range=None, y_range=None):

  dims = getDims(fName)
  if dims.__len__() == 2:
    jdm, idm = dims
  else:
    jdm, idm, kdm = dims
    print("\n-- CAUTION! Read 3d archive is not yet ready!\n")

  reclen = 4*idm*jdm # Record length in bytes
  # HYCOM binary data is written out in chunks/"words" of multiples of 4096*4 bytes.
  # Length of one level of one variable (reclen) will be between
  # consecutive multiples of the wordlen. Data is padded to bring the volume
  # up to the next multiple. The "pad" value below is equal to the bytes that are needed to do this.
  wordlen = 4096*4
  pad = wordlen * np.ceil(reclen / wordlen) - reclen   # Pad size in bytes
  fieldRecords = getFieldIndex(field,fName)         # Get field record indices
  fieldAddresses = np.array(fieldRecords)*(reclen+pad) # Address in bytes

  file = open_a_file(fName,mode='rb') # Open file
  if dims.__len__() == 2: # 2-d field
    field = np.zeros((jdm,idm))
    file.seek(int(fieldAddresses[0]),0) # Move to address of the field
    data = file.read(idm*jdm*4)
    field = np.reshape(np.frombuffer(data, dtype='float32', count=idm*jdm),(jdm,idm)).byteswap()

    if not x_range is None:
      field = field[:,:,x_range]
    if not y_range is None:
      field = field[:,y_range,:]

  #field = field.byteswap() # Convert to little-endian
  file.close()
  field[field == np.float32(2**100)] = undef

  return field

In [ ]:
# Number of records in the binary file, read from .b
def getNumberOfRecords(fName):
  f = getTextFile(get_b_filename(fName))
  if 'arch' in fName:
      f = f[10:]; return len(f)
  if 'grid' in fName:
      f = f[3:]; return len(f)
  if 'depth' in fName:
      return 1
  if 'restart' in fName:
      f = f[2:]; return len(f)

In [ ]:
def getBathymetry(grid_fName, topog_fName, undef=np.nan):

  jdm,idm = getDims(grid_fName)

  file = open_a_file(topog_fName, mode='rb')
  #Data is in float32, which has 4 bytes/value
  data = file.read(idm*jdm*4)
  field = np.reshape(np.frombuffer(data,dtype='float32',count=idm*jdm).byteswap(),(jdm,idm))
  file.close()

  print(f"field.shape={field.shape}")
  field[field>2**99] = undef

  return field

In [ ]:
bFile=get_b_filename(data_path+"/regional.grid.a")
print(f"Reading {bFile}\n")

jm, im = getDims(data_path+"/regional.grid.a")
print(f"im={im}, jm={jm}")

jm, im = getDims(data_path+"/depth_GLBb0.08_09m11.a", topo_file=True)
print(f"im={im}, jm={jm}")

In [ ]:
fn1 = data_path+"/regional.grid.a"
print(getFieldIndex('plat',fn1))
print("\n")

fn2 = data_path+"/v2.4_rtofs_glo.t00z.n00.archs.a"
print(getFieldIndex('srfhgt',fn2))
print(getFieldIndex('salin',fn2))

In [ ]:
print(getNumberOfRecords(data_path+"/regional.grid.a"))
print("\n")
print(getNumberOfRecords(data_path+"/depth_GLBb0.08_09m11.a"))
print("\n")
print(getNumberOfRecords(data_path+"/v2.4_rtofs_glo.t00z.n00.archs.a"))

In [ ]:
depth=getBathymetry(data_path+"/regional.grid.a", data_path+"/depth_GLBb0.08_09m11.a")

In [ ]:
plt.imshow(depth, origin='lower')
plt.colorbar()

In [ ]:
plon = getField('plon', data_path+"/regional.grid.a")
plat = getField('plat', data_path+"/regional.grid.a")

# variables
sst = getField('temp',   data_path+"/v2.4_rtofs_glo.t00z.n00.archs.a")
sss = getField('salin',  data_path+"/v2.4_rtofs_glo.t00z.n00.archs.a")
ssh = getField('srfhgt', data_path+"/v2.4_rtofs_glo.t00z.n00.archs.a")

uvel = getField('u-vel', data_path+"/v2.4_rtofs_glo.t00z.n00.archs.a")
vvel = getField('v-vel', data_path+"/v2.4_rtofs_glo.t00z.n00.archs.a")

In [ ]:
# SSH in m
grav = 9.81
ssh = ssh/grav #https://github.com/HYCOM/HYCOM-src/blob/5c9f48918374965e2df09042180ea8a69c650dc4/mod_cb_arrays.F90#L90

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(2,2,1)
plt.imshow(sst, origin='lower'); plt.colorbar(shrink=0.4)

plt.subplot(2,2,2)
plt.imshow(sss, origin='lower'); plt.colorbar(shrink=0.4)

plt.subplot(2,2,3)
plt.imshow(ssh, origin='lower'); plt.colorbar(shrink=0.4)

plt.subplot(2,2,4)
plt.imshow( np.sqrt(uvel**2 + vvel**2), origin='lower'); plt.colorbar(shrink=0.4)

In [ ]:
!cat $data_path/v2.4_rtofs_glo.t00z.n00.archs.b

## Create (xarray) dataset to simply life!

In [ ]:
# convert time stamp in archive to yyyy/mm/dd:hh:mm:ss
# translation of https://github.com/HYCOM/HYCOM-tools/blob/master/bin/hycom_wind_ymdh.f

def hycom_wind_ymdh(dtime, yrflag=3):

  if yrflag !=3:
    raise ValueError("yrflag must be 3")

  [iyear, month, iday, ihour] = fordate(dtime, yrflag)

  return iyear, month, iday, ihour

def fordate(dtime, yrflag=3):

  if yrflag !=3:
    raise ValueError("yrflag must be 3")

  month0=np.asarray([[1,  31,  61,  91, 121, 151, 181, 211, 241, 271, 301, 331, 361],
               [1,  32,  60,  91, 121, 152, 182, 213, 244, 274, 305, 335, 366],
               [1,  32,  61,  92, 122, 153, 183, 214, 245, 275, 306, 336, 367]])

  [iyear, jday, ihour] = forday(dtime, yrflag)

  if (np.mod(iyear,4) == 0):
    k = 3  #leap year
  else:
    k = 2  #standard year

  for m in range(0, 12):
    #print(jday, month0[k-1, m], month0[k-1, m+1])
    if ((jday >= month0[k-1, m]) and (jday < month0[k-1, m+1])):
      month = m+1
      iday  = jday - month0[k-1, m] + 1
  #print(month, iday)

  return iyear, month, iday, ihour

def forday(dtime, yrflag=3): # yrflag is always set to 3.
  """
  # Follow @awallcraft's logic to undo hycom_wind_date conversion
  # https://github.com/HYCOM/HYCOM-tools/blob/3647ab0efbce59bcb2d8d6b3f66978b4a661b6b6/bin/hycom_wind_date.f#L54
  """

  if yrflag !=3:
    raise ValueError("yrflag must be 3")

  # model day is calendar days since 01/01/1901
  iyr   = (dtime-1.)/365.25
  nleap = iyr/4.
  dtim1 = 365.0*iyr + nleap + 1.
  day   = dtime - dtim1 + 1.
  #print(iyr, nleap, dtim1, day)

  if dtim1 > dtime:
    iyr = iyr - 1
  elif day >= 367.:
    iyr = iyr + 1
  elif ((day >= 366.0) and (np.mod(iyr,4) != 3)):
    iyr = iyr + 1

  nleap = iyr/4
  dtim1 = 365.0*iyr + nleap + 1.
  #print(iyr, nleap, dtim1)

  iyear =  1901 + iyr
  iday  =  dtime - dtim1 + 1
  ihour = (dtime - dtim1 + 1.001 - iday)*24.
  #print(iyear, iday, ihour)

  return iyear, iday, ihour

In [ ]:
dummy_date = '2099-01-01'

#print(hycom_wind_ymdh(45370.000)) # 2025/03/20
#print(hycom_wind_ymdh(45382.000)) # 2025/04/01

In [ ]:
ds = xr.Dataset()

ds['Latitude'] = xr.DataArray(plat,
    dims=("Y", "X"),
    coords={"Y": np.arange(1, plat.shape[0]+1), "X": np.arange(1, plat.shape[1]+1)},
    name="Latitude",
    attrs={"units": "degrees_north", "long_name": "latitude"})

ds['Longitude'] = xr.DataArray(plon,
    dims=("Y", "X"),
    coords={"Y": np.arange(1, plon.shape[0]+1), "X": np.arange(1, plon.shape[1]+1)},
    name="Longitude",
    attrs={"units": "degrees_east", "long_name": "longitude", "modulo": "360 degrees"})

ds['time'] = np.array([str(dummy_date)], dtype='datetime64')

ds['srfhgt'] = xr.DataArray(ssh,
    dims=("Y", "X"),
    coords={"Y": np.arange(1, ssh.shape[0]+1), "X": np.arange(1, ssh.shape[1]+1)},
    name="srfhgt",
    attrs={"units": "m", "long_name": "surface height"})

ds.attrs = {"source": "NCEP RTOFS v2.5",
            "history": "converted archive to netcdf",
            "using": "https://github.com/NOAA-EMC/RTOFS_GLO"}

ds=ds.set_coords(["Latitude", "Longitude", "time"])

In [ ]:
ds

## Plot few regions/projections:
- N Pole (Arctic).
- Globe.
- S Pole (Antarctic).

In [ ]:
fig = plt.figure(figsize=[8,6])

ax = fig.add_subplot(1,1,1, projection=ccrs.NorthPolarStereo(central_longitude=-30.0))
ax.add_feature(cfeature.LAND, facecolor='grey', alpha=0.1)
ax.coastlines(color='k', alpha=0.1)
ax.set_extent([-300, 60, 50, 90], ccrs.PlateCarree())

im = ds.srfhgt.plot(ax=ax, x='Longitude', y='Latitude',
            vmin=-1, vmax=1, cmap='bwr',
            transform=ccrs.PlateCarree(),
            add_labels=False, add_colorbar=False)

im.axes.gridlines(color='black', alpha=0.5, linestyle='--', draw_labels=True)

cax = ax.inset_axes([0.8, 0.3, 0.03, 0.6])
fig.colorbar(im, cax=cax, orientation='vertical', ticks=[-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.])
cax.tick_params(labelsize=10, rotation=0)
cax.set_title('{} [{}]'.format(ds.srfhgt.attrs['long_name'], ds.srfhgt.attrs['units']))
#cax.set_title('{}/{}/{}'.format(ds.time.dt.year.values[0], ds.time.dt.month.values[0], ds.time.dt.day.values[0]))

In [ ]:
fig = plt.figure(figsize=[8,6])

ax = fig.add_subplot(1,1,1, projection=ccrs.Miller(central_longitude=-180.0))
ax.add_feature(cfeature.LAND, facecolor='grey', alpha=0.1)
ax.coastlines(color='k', alpha=0.1)
#ax.set_extent([-300, 60, -60, 60], ccrs.PlateCarree())

im = ds.srfhgt.plot(ax=ax, x='Longitude', y='Latitude',
            vmin=-1, vmax=1, cmap='bwr',
            transform=ccrs.PlateCarree(),
            add_labels=False, add_colorbar=False)

im.axes.gridlines(color='black', alpha=0.5, linestyle='--', draw_labels=True)

cax = ax.inset_axes([0.02, 0.05, 0.4, 0.03])
fig.colorbar(im, cax=cax, orientation='horizontal', ticks=[-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.])
cax.tick_params(labelsize=8, rotation=0)
cax.set_title('{} [{}]'.format(ds.srfhgt.attrs['long_name'], ds.srfhgt.attrs['units']))
#cax.set_title('{}/{}/{}'.format(ds.time.dt.year.values[0], ds.time.dt.month.values[0], ds.time.dt.day.values[0]))

In [ ]:
fig = plt.figure(figsize=[8,6])

ax = fig.add_subplot(1,1,1, projection=ccrs.SouthPolarStereo(central_longitude=-120.0))
ax.add_feature(cfeature.LAND, facecolor='grey', alpha=0.1)
ax.coastlines(color='k', alpha=0.1)
ax.set_extent([-300, 60, -40, -90], ccrs.PlateCarree())

im = ds.srfhgt.plot(ax=ax, x='Longitude', y='Latitude',
                  vmin=-1, vmax=1, cmap='bwr',
                  transform=ccrs.PlateCarree(),
                 add_labels=False, add_colorbar=False)

im.axes.gridlines(color='black', alpha=0.5, linestyle='--', draw_labels=True)

cax = ax.inset_axes([0.3, 0.45, 0.35, 0.03])
fig.colorbar(im, cax=cax, orientation='horizontal', ticks=[-1, -0.75, -0.5, -0.25, 0, 0.25, 0.5, 0.75, 1.])
cax.tick_params(labelsize=6, rotation=30)
cax.set_title('{} [{}]'.format(ds.srfhgt.attrs['long_name'], ds.srfhgt.attrs['units']))
##cax.set_title('{}/{}/{}'.format(ds.time.dt.year.values[0], ds.time.dt.month.values[0], ds.time.dt.day.values[0]))

## Read time stamp in archive [b] file

In [ ]:
def get_model_day(fName, varName):
  bFile = getTextFile( get_b_filename(fName))
  for line in bFile:
    if varName in line:
      model_day = line.split()[3]
  print(model_day)

In [ ]:
get_model_day(data_path+"/v2.4_rtofs_glo.t00z.n00.archs.a", 'srfhgt')